from here: https://github.com/keras-team/keras/blob/master/examples/mnist_cnn.py

https://blog.keras.io/keras-as-a-simplified-interface-to-tensorflow-tutorial.html#using-keras-models-with-tensorflow

https://github.com/keras-team/keras/issues/5427
    

In [1]:
'''Trains a simple convnet on the MNIST dataset.

Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import tensorflow as tf
import numpy as np

Using TensorFlow backend.


In [2]:
batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))


def cohens_kappa(y_true, y_pred):
    y_true_classes = tf.argmax(y_true, 1)
    y_pred_classes = tf.argmax(y_pred, 1)
    return tf.contrib.metrics.cohen_kappa(y_true_classes, y_pred_classes, 10)[1]


def tf_accuracy(y_true, y_pred):
    y_true_classes = tf.argmax(y_true, 1)
    y_pred_classes = tf.argmax(y_pred, 1)
    # preds and targets are switched in tf.contrib.metrics.accuracy
    return tf.contrib.metrics.accuracy(y_pred_classes, y_true_classes)
    

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy', cohens_kappa, tf_accuracy])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples


In [3]:
# tensorflow variables need to be initialized before calling model.fit()
# with tf.Session() as sess:
# K.get_session().run(tf.global_variables_initializer())
K.get_session().run(tf.local_variables_initializer())

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))

score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 5s 90us/step - loss: 0.2686 - acc: 0.9177 - cohens_kappa: 0.8037 - tf_accuracy: 0.9177 - val_loss: 0.0635 - val_acc: 0.9795 - val_cohens_kappa: 0.9134 - val_tf_accuracy: 0.9795
Epoch 2/12
60000/60000 [==============================] - 4s 68us/step - loss: 0.0916 - acc: 0.9725 - cohens_kappa: 0.9318 - tf_accuracy: 0.9725 - val_loss: 0.0397 - val_acc: 0.9858 - val_cohens_kappa: 0.9434 - val_tf_accuracy: 0.9858
Epoch 3/12
60000/60000 [==============================] - 4s 69us/step - loss: 0.0681 - acc: 0.9800 - cohens_kappa: 0.9504 - tf_accuracy: 0.9800 - val_loss: 0.0436 - val_acc: 0.9846 - val_cohens_kappa: 0.9554 - val_tf_accuracy: 0.9846
Epoch 4/12
60000/60000 [==============================] - 4s 68us/step - loss: 0.0540 - acc: 0.9835 - cohens_kappa: 0.9592 - tf_accuracy: 0.9835 - val_loss: 0.0328 - val_acc: 0.9899 - val_cohens_kappa: 0.9623 - val_tf_accuracy: 0.9899
Epoch 5/12

In [4]:
# confirming the correct kappa score for debugging
from sklearn.metrics import cohen_kappa_score

cohen_kappa_score(np.array([1, 2]), np.array([0, 2]))

0.33333333333333337

In [5]:
# Here are some examples of tensorflow debugging

sess = tf.Session()

# K.get_session().run(tf.local_variables_initializer())
# K.get_session().run(tf.global_variables_initializer())

yt_tens = tf.convert_to_tensor(np.array([[0, 1, 0], [0, 0, 1]]))
yp_tens = tf.convert_to_tensor(np.array([[1, 0, 0], [0, 0, 1]]))

wh = tf.where(tf.equal(yt_tens, 1))[:, 1]
wh2 = tf.where(tf.equal(yp_tens, 1))[:, 1]
print(sess.run(yt_tens), '\n')
print(sess.run(wh), '\n')
print(sess.run(wh2), '\n')

# returns the value and an update_op object.  The update_op object appears to have the correct value;
# the other one seems to be 0.
ck = tf.contrib.metrics.cohen_kappa(tf.where(tf.equal(yt_tens, 1))[:, 1],
                                    tf.where(tf.equal(yp_tens, 1))[:, 1],
                                    10)[1]

# need to initialize vars for cohens kappa
# needs to be run after creating cohens kappa line
sess.run(tf.global_variables_initializer())
sess.run(tf.local_variables_initializer())

# should match the cohens kappa calc from sklearn above
print(sess.run(ck))

[[0 1 0]
 [0 0 1]] 

[1 2] 

[0 2] 

0.3333333333333333
